In [1]:
import re
from importlib.util import spec_from_file_location, module_from_spec
import os
from collections import defaultdict
import pandas as pd

In [2]:
def experiment_params(def_path: str):
    lines = open(def_path).readlines()
    code = "".join(lines)
    defs = re.findall("(def\s.*?\):)", code, re.DOTALL)
    paramdict = {
        re.findall("def\s(.*)\(", d)[0]: [
            y
            for y in [x.strip(": #") for x in re.findall("(.*\:)", d)[:-1]]
            if not y.startswith('"') and "(" not in y
        ]
        for d in defs
    }
    funs = [re.findall("def\s(.*)\(", d)[0] for d in defs]
    starts = [[x.strip() for x in lines].index(d.split("\n")[0].strip()) for d in defs]
    starts.append(None)
    omit_params = ["experiment", "experiment_version"]
    fund = {}
    for i, f in enumerate(funs):
        block = "".join(lines[starts[i] : starts[i + 1]])
        adds = re.findall("apm\.add\((.*?){", block, re.DOTALL)
        actservs = re.findall('"action_server":(.*),', block)  # [0].strip(' "')
        actnames = re.findall('"action_name":(.*),', block)  # [0].strip(' "')
        actlist = []
        if actservs:
            for s, a in zip(actservs, actnames):
                actlist.append(
                    s.strip().replace("_server", "")
                    + "___"
                    + a.strip().strip("'").strip('"')
                )
        if adds:
            for a in adds:
                actlist.append(
                    a.split("_server")[0].strip()
                    + "___"
                    + a.split("_server")[-1].strip(" ,\n").strip().strip("'").strip('"')
                )
        if actlist:
            fund[f] = sorted(
                [
                    x
                    for x in actlist
                    if not any([x.endswith(omit) for omit in omit_params])
                ]
            )
    return fund, sorted(
        set(
            [
                f"{k}___{x}"
                for k, v in paramdict.items()
                for x in v
                if x not in omit_params
            ]
        )
    )

In [3]:
def action_params(def_path: str):
    lines = open(def_path).readlines()
    code = "".join(lines)
    defs = re.findall("(def\s.*?\):)", code, re.DOTALL)
    paramdict = {
        re.findall("def\s(.*)\(", d)[0]: [
            y
            for y in [x.strip(": #") for x in re.findall("(.*\:)", d)[:-1]]
            if not y.startswith('"') and "(" not in y
        ]
        for d in defs
    }
    omit_params = ["action", "action_version", "A"]
    return sorted(
        set(
            [
                f"{k}___{x}"
                for k, v in paramdict.items()
                for x in v
                if x not in omit_params
            ]
        )
    )

In [4]:
def validate_params(config_path: str):
    conf_spec = spec_from_file_location("config", config_path)
    conf_mod = module_from_spec(conf_spec)
    conf_spec.loader.exec_module(conf_mod)
    config = conf_mod.config

    exp_acts = {}
    all_exp = {}
    for el in config["experiment_libraries"]:
        aset, pset = experiment_params(
            f"{os.sep}".join(
                os.path.abspath(config_path).split(os.sep)[:-2]
                + ["experiment", f"{el}.py"]
            )
        )
        exp_acts[el] = aset
        all_exp[el] = pset

    all_act = {}
    for skey, sdict in config["servers"].items():
        if "fast" in sdict.keys():
            pset = action_params(
                f"{os.sep}".join(
                    os.path.abspath(config_path).split(os.sep)[:-2]
                    + ["server", sdict["group"], f"{sdict['fast']}.py"]
                )
            )
            all_act[skey] = pset

    return exp_acts, all_exp, all_act

In [5]:
def generate_param_df(config_path: str):
    
    ea, ae, aa = validate_params(config_path)
    process_params = defaultdict(set)

    for ekey, edict in ea.items():
        for ename, alist in edict.items():
            all_exp_params = ae[ekey]
            for ep in all_exp_params:
                try:
                    epk, epp = ep.split("___")
                except ValueError:
                    print(ep)
                if ename == epk:
                    process_params[epp].add((tuple(['__EXP__', ename, ekey])))
            for act_str in alist:
                akey, aname = act_str.split("___")
                act_params = [x.split("___")[-1] for x in aa[akey] if x.split("___")[0]==aname]
                for p in act_params:
                    process_params[p].add(tuple([f"{akey}___{aname}", ename, ekey]))

    param_df = pd.concat([pd.DataFrame(v, index=[k]*len(v)) for k,v in process_params.items()]).reset_index()
    param_df.columns = ['parameter_name', 'action', 'experiment', 'exp.py']
    
    return param_df

In [6]:
def get_all_params(config_dir: str):
    omit_confs = ['modelyst_dev.py', 'ros_dev.py', '__init__.py']
    all_configs = [os.path.join(config_dir, x) for x in os.listdir(config_dir) if x not in omit_confs and x.endswith(".py")]
    alldfs = []
    for conf in all_configs:
        try:
            df = generate_param_df(conf)
            alldfs.append(df)
        except:
            print(conf)
    return pd.concat(alldfs).drop_duplicates().sort_values(['parameter_name', 'experiment', 'action'])

In [8]:
paramsdf = get_all_params("../config")

In [9]:
paramsdf.groupby(["parameter_name", "exp.py"]).aggregate(lambda x: sorted(set(x)))

action  \
parameter_name  exp.py                                                           
AcqInterval__s  ADSS_exp                     [PSTAT___run_CA, PSTAT___run_OCV]   
                ANEC_exp                      [PSTAT___run_CA, PSTAT___run_CV]   
                SDC_exp       [PSTAT___run_CA, PSTAT___run_CP, PSTAT___run_CV]   
                simulate_exp                                  [PSTAT___run_CP]   
CA_duration_sec ADSS_exp                                             [__EXP__]   
...                                                                        ...   
x_mm            ADSS_exp                                             [__EXP__]   
                SDC_exp                                              [__EXP__]   
y_mm            ADSS_exp                                             [__EXP__]   
                SDC_exp                                              [__EXP__]   
z_move_mm       ANEC_exp                                             [__EXP__]   

                                                                     experiment  
parameter_name  exp.py                                                           
AcqInterval__s  ADSS_exp                                 [ADSS_slave_CA, debug]  
                ANEC_exp      [ANEC_slave_CA, ANEC_slave_CV, ANEC_slave_phot...  
                SDC_exp       [SDC_slave_CA, SDC_slave_CA_led, SDC_slave_CP,...  
                simulate_exp                                   [SIM_measure_CP]  
CA_duration_sec ADSS_exp                                        [ADSS_slave_CA]  
...                                                                         ...  
x_mm            ADSS_exp                            [ADSS_slave_startup, debug]  
                SDC_exp                                        [SDC_slave_move]  
y_mm            ADSS_exp                            [ADSS_slave_startup, debug]  
                SDC_exp                                        [SDC_slave_move]  
z_move_mm       ANEC_exp                                   [ANEC_slave_startup]  

[252 rows x 2 columns]

In [10]:
paramsdf.groupby(["parameter_name"]).aggregate(lambda x: sorted(set(x)))

,action,experiment,exp.py
parameter_name,,,
AcqInterval__s,"[PSTAT___run_CA, PSTAT___run_CP, PSTAT___run_C...","[ADSS_slave_CA, ANEC_slave_CA, ANEC_slave_CV, ...","[ADSS_exp, ANEC_exp, SDC_exp, simulate_exp]"
CA_duration_sec,[__EXP__],"[ADSS_slave_CA, ANEC_slave_CA, ANEC_slave_phot...","[ADSS_exp, ANEC_exp, SDC_exp]"
CA_potential,[__EXP__],[ADSS_slave_CA],[ADSS_exp]
CA_potential_vsRHE,[__EXP__],"[SDC_slave_CA, SDC_slave_CA_led]",[SDC_exp]
CP_current,[__EXP__],"[SDC_slave_CP, SDC_slave_CP_led, SDC_slave_bac...",[SDC_exp]
...,...,...,...
wavelength_intensity_mw,[__EXP__],"[SDC_slave_CA_led, SDC_slave_CP_led, SDC_slave...",[SDC_exp]
wavelength_nm,[__EXP__],"[SDC_slave_CA_led, SDC_slave_CP_led, SDC_slave...",[SDC_exp]
x_mm,[__EXP__],"[ADSS_slave_startup, SDC_slave_move, debug]","[ADSS_exp, SDC_exp]"
